<a href="https://colab.research.google.com/github/Basu8971/soil-health-map/blob/main/Soil%20health%20maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import necessary libraries
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import numpy as np
import folium
from branca.colormap import linear
















# Authenticate the Google account
auth.authenticate_user()
















# Use google-auth to get credentials and authorize gspread
creds, _ = default()
gc = gspread.authorize(creds)
















# Replace this URL with the correct URL of your Google Sheets
sheet_1_url = 'https://docs.google.com/spreadsheets/d/1s0G455slsFyyqDiS2a5rcxWz0xpAlI91aR9UFhVKRgs/edit?gid=0'
















# Open the Google Sheet using gspread
sheet_1 = gc.open_by_url(sheet_1_url)
















# Load the data from the sheet into a DataFrame
worksheet = sheet_1.sheet1  # Get the first sheet
df = pd.DataFrame(worksheet.get_all_records())
















# Replace empty cells (e.g., '' or '-') with NaN
df.replace(['', '-'], np.nan, inplace=True)
















# Display the first few rows of the dataset to ensure it's loaded correctly
print("First DataFrame (from Sheet 1):")
print(df.head())
















# Directly use the existing Latitude and Longitude columns
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
















# Custom rainbow colors for different categories
rainbow_colors = [
    '#FF0000',  # Red
    '#FF7F00',  # Orange
    '#FFFF00',  # Yellow
    '#00FF00',  # Green
    '#0000FF',  # Blue
    '#4B0082',  # Indigo
    '#8B00FF'   # Violet
]






# Soil texture types and their corresponding colors
soil_texture_colors = {
    'Silty Clay Loam': '#FFFF00',  # Bright Yellow
    'Sandy Clay Loam': '#FF7F00',  # Bright Orange
    'Silty Clay': '#FF1493',       # Bright Pink
    'Sandy Clay': '#32CD32',       # Bright Green
    'Silt Loam': '#00FFFF',        # Bright Cyan
    'Clay loam': '#FF4500',        # Bright Red
    'Clay': '#8A2BE2',             # Bright Purple
    'Sandy Loam': '#1E90FF',       # Bright Blue
    'Loamy Sand': '#FF00FF'        # Bright Magenta
}








# Define the property ranges for the legend (based on new requirements)
property_ranges = {
    'Soil organic carbon concentration(%)': [0, 1.0],  # Ratings: 0-1.0
    'Soil organic carbon content (Stock- tonnes/ha)': [0, 100],  # Ratings: 0-100
    'Carbon mineralization potential (mgCO2/kg)': [0, 2000],  # Ratings: 0-2000
    'Presence of invertebrates': ['Poor', 'Fair', 'Good'],  # Categories
    'Soil Aggregate Stability': ['Poor', 'Fair', 'Good'],  # Categories
    'Soil pH': [0, 14],  # Soil pH range (0 to 14)
    'Soil Structure': ['Poor', 'Fair', 'Good'],
    'Soil texture': None,  # Allow dynamic handling of soil texture values
}
























# Create the map centered at the mean latitude and longitude
m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=10)










# Create the map centered at India (Latitude: 20.5937, Longitude: 78.9629)
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)  # Zoom level set to 5 for India




# Add Esri World Imagery Satellite Layer
folium.TileLayer(
    tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Esri',
    name='Esri Satellite',
    overlay=False,
    control=True
).add_to(m)
















def add_property_layer(property_column, layer_name, m, is_numerical=True):
    # Ensure that "Manrope" font is available in the map's HTML
    m.get_root().html.add_child(folium.Element("""
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Manrope&display=swap');
            .popup-text {
                font-family: 'Manrope', sans-serif;
                font-size: 14px;
                line-height: 1.6;
            }
        </style>
    """))


    # Special case handling for Soil pH
    if property_column == 'Soil pH':
        categories = ['Acidic', 'Medium', 'Alkaline', 'Fatal']
        colors = ['#FF0000', '#FFFF00', '#00FF00', '#0000FF']  # Red for Acidic, Yellow for Medium, Green for Alkaline, Blue for Fatal


        # Create the FeatureGroup for this property
        property_layer = folium.FeatureGroup(name=layer_name)


        # Iterate over the dataframe rows to create markers
        for _, row in df.iterrows():
            value = row[property_column]


            # Skip rows where the value is NaN or missing
            if pd.isna(value):
                continue


            # Classify the Soil pH value and determine the color
            if value < 7:
                category = 'Acidic'
                color = colors[0]
            elif 7 <= value <= 8.5:
                category = 'Medium'
                color = colors[1]
            elif 8.51 <= value <= 9:
                category = 'Alkaline'
                color = colors[2]
            else:
                category = 'Fatal'
                color = colors[3]


            # Dynamically adjust the radius based on the pH value (for example: scale it between 5 and 20)
            radius = 5 + (value * 2)  # This scales the pH value to a range of circle sizes


            # Create a popup with the farmer's name and pH value
            popup_text = f"""
                <div class="popup-text">
                    <ul style="list-style-type: disc; margin-left: 20px;">
                        <li><b>Farmer Code:</b> {row['Farmer code']}</li>
                        <li><b>{property_column}:</b> {value} ({category})</li>
                    </ul>
                </div>
            """


            # Create circle markers with a popup
            folium.CircleMarker(
                location=[row['Latitude'], row['Longitude']],
                radius=radius,
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.7,
                popup=folium.Popup(popup_text, max_width=300)
            ).add_to(property_layer)


        property_layer.add_to(m)


    else:
        # Handle other properties (numerical or categorical)
        if is_numerical:
            # Get the property range (either numerical or categorical)
            min_value, max_value = property_ranges[property_column]
            range_diff = max_value - min_value
            num_categories = 4  # You can change this based on your range (3, 4, etc.)
            colors = rainbow_colors[:num_categories]  # Use first N colors of rainbow


            # Define the intervals for the categories
            intervals = [min_value + (range_diff / num_categories) * i for i in range(num_categories + 1)]


            # Create the FeatureGroup for this property
            property_layer = folium.FeatureGroup(name=layer_name)


            # Iterate over the dataframe rows to create markers
            for _, row in df.iterrows():
                value = row[property_column]


                # Skip rows where the value is NaN or missing
                if pd.isna(value):
                    continue


                try:
                    value = float(value)  # Ensure value is numeric
                except ValueError:
                    continue  # Skip if the value is not numeric (e.g., if it's a string)


                # Determine which color to use based on the value
                for i in range(num_categories):
                    if value >= intervals[i] and value < intervals[i + 1]:
                        color = colors[i]
                        break


                # Dynamically adjust the radius based on the value (min-max scale between 5 and 20)
                radius = 5 + ((value - min_value) / range_diff) * 15  # Scale the value to a radius range


                # Create circle markers with a popup to show the farmer's name and parameter values
                popup_text = f"""
                    <div class="popup-text">
                        <ul style="list-style-type: disc; margin-left: 20px;">
                            <li><b>Farmer Code:</b> {row['Farmer code']}</li>
                            <li><b>{property_column}:</b> {value}</li>
                        </ul>
                    </div>
                """
                folium.CircleMarker(
                    location=[row['Latitude'], row['Longitude']],
                    radius=radius,  # Radius dynamically adjusted based on value
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=folium.Popup(popup_text, max_width=300)
                ).add_to(property_layer)


            property_layer.add_to(m)


        else:
            # Handle categorical data (e.g., Soil Aggregate Stability, Presence of Invertebrates, etc.)
            if property_column == 'Soil Aggregate Stability':
                categories = ['Poor', 'Fair', 'Good']
                colors = ['#FF0000', '#FFFF00', '#00FF00']  # Red for Poor, Yellow for Fair, Green for Good
            elif property_column == 'Presence of invertebrates':
                categories = ['Poor', 'Fair', 'Good']
                colors = ['#FF0000', '#FFFF00', '#00FF00']  # Red for Poor, Yellow for Fair, Green for Good
            elif property_column == 'Soil texture':
                # Special handling for Soil texture with predefined categories and colors
                categories = ['Silty Clay Loam', 'Sandy Clay Loam', 'Silty Clay', 'Sandy Clay',
                              'Silt Loam', 'Clay loam', 'Clay', 'Sandy Loam', 'Loamy Sand']
                colors = [
                    '#FFFF00',  # Silty Clay Loam (Bright Yellow)
                    '#FF7F00',  # Sandy Clay Loam (Bright Orange)
                    '#FF1493',  # Silty Clay (Bright Pink)
                    '#32CD32',  # Sandy Clay (Bright Green)
                    '#00FFFF',  # Silt Loam (Bright Cyan)
                    '#FF4500',  # Clay loam (Bright Red)
                    '#8A2BE2',  # Clay (Bright Purple)
                    '#1E90FF',  # Sandy Loam (Bright Blue)
                    '#FF00FF'   # Loamy Sand (Bright Magenta)
                ]


            else:
                categories = property_ranges[property_column]
                colors = rainbow_colors[:len(categories)]


            # Create the FeatureGroup for this property
            property_layer = folium.FeatureGroup(name=layer_name)


            # Iterate over the dataframe rows to create markers
            for _, row in df.iterrows():
                value = row[property_column]


                # Skip rows where the value is NaN or missing
                if pd.isna(value):
                    continue


                # Handle missing categories by checking if the value is in the categories
                if value not in categories:
                    print(f"Warning: {value} not found in categories {categories}. Skipping this row.")
                    continue  # Skip this marker


                # Find the category index based on value
                category_index = categories.index(value)
                color = colors[category_index]


                # Check if the property is 'Soil texture' and assign a fixed radius
                if property_column == 'Soil texture':
                    radius = 8  # Fixed radius for soil texture
                else:
                    # Dynamically adjust the radius for other properties
                    radius = 5 + (category_index * 5)  # Example of scaling the radius for categorical data


                # Create circle markers with a popup to show the farmer's name and parameter values
                popup_text = f"""
                    <div class="popup-text">
                        <ul style="list-style-type: disc; margin-left: 20px;">
                            <li><b>Farmer Code:</b> {row['Farmer code']}</li>
                            <li><b>{property_column}:</b> {value}</li>
                        </ul>
                    </div>
                """
                folium.CircleMarker(
                    location=[row['Latitude'], row['Longitude']],
                    radius=radius,  # Radius dynamically adjusted based on category
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=folium.Popup(popup_text, max_width=300)
                ).add_to(property_layer)


            property_layer.add_to(m)
















# Add the property layers for each variable (corrected spelling of layer names)
add_property_layer('Soil organic carbon concentration(%)', 'Chemical Property- Soil Organic Carbon', m)
add_property_layer('Soil pH', 'Chemical Property- Soil pH', m, is_numerical=True)
add_property_layer('Soil organic carbon content (Stock- tonnes/ha)', 'Physical Property- Soil Organic Carbon Stock', m)
add_property_layer('Soil Structure', 'Physical Property- Soil Structure', m, is_numerical=False)
add_property_layer('Soil texture', 'Physical Property- Soil Texture', m, is_numerical=False)
add_property_layer('Soil Aggregate Stability', 'Physical Property- Soil Aggregate Stability', m, is_numerical=False)
add_property_layer('Carbon mineralization potential (mgCO2/kg)', 'Biological Property- Carbon Mineralization Potential', m)
add_property_layer('Presence of invertebrates', 'Biological Property- Presence of Invertebrates', m, is_numerical=False)








# Correctly treat as categorical








# Modify the column names in the map
for column in ['Bacteria(cfu/g)', 'Actinomycetes(cfu/g)', 'Fungi(cfu/g)']:
    # Modify the label to reflect the soil microbial diversity naming convention
    new_column_name = f"Soil microbial diversity- {column.split('(')[0].strip()}"

    # Create the feature layer for the column with the new name (without (cfu/g))
    property_layer = folium.FeatureGroup(name=new_column_name)

    for _, row in df.iterrows():
        value = row[column]
        if pd.isna(value):
            continue

        # Modify popup text to use bullet points format, keeping (cfu/g) in the popup
        popup_text = f"""
            <ul style="list-style-type: disc; margin-left: 20px;">
                <li><b>Farmer Code:</b> {row['Farmer code']}</li>
                <li><b>{new_column_name}:</b> {value} (cfu/g)</li>  <!-- Keep the unit in the popup -->
            </ul>
            """

        # Create circle markers for each row of the data
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=10,  # Radius in pixels (will scale with zoom)
            color='#0000FF',  # Default blue color for no categorization
            fill=True,
            fill_color='#0000FF',
            fill_opacity=0.7,
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(property_layer)

    property_layer.add_to(m)
















# Add Layer Control to toggle between layers
folium.LayerControl(position='topright', collapsed=True).add_to(m)
















def create_property_legends():
    # Add the link to the Manrope font
    legend_html = """
    <style>
        /* Load Manrope font */
        @import url('https://fonts.googleapis.com/css2?family=Manrope:wght@400;500;600&display=swap');

        /* Apply Manrope font to the entire map text */
        body, div, span, h4, b, ul, li, p {
            font-family: 'Manrope', sans-serif !important;
        }
    </style>

    <div style="position: fixed; top: 10px; left: 10px; width: 150px; z-index: 9999; padding: 5px; background-color: rgba(255, 255, 255, 0.8); border-radius: 5px; font-size: 8px;">
        <h4 style="text-align: center; font-size: 10px; margin: 3px 0;">Soil Health Parameters</h4>
    """
    # Legend for Soil Organic Carbon Concentration (%)
    legend_html += """
    <div>
        <b>Soil Organic Carbon (%)</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0 - 0.2 Very Low
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF7F00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0.21 - 0.40 Low
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0.41 - 0.60 Medium
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0.61 - 0.80 High
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #0000FF; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0.81 - 1.0 Very High
        </div>
    </div>
    """

    # Legend for Soil Organic Carbon (tonnes/ha)
    legend_html += """
    <div>
        <b>Soil Organic Carbon (tonnes/ha)</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0 - 25 Low
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF7F00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 26 - 50 Medium
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 51 - 75 High
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 76 - 100 Very High
        </div>
    </div>
    """

    # Legend for Carbon Mineralization (mgCO2/kg)
    legend_html += """
    <div>
        <b>Carbon Mineralization (mgCO2/kg)</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 0 - 500 Low
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF7F00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 501 - 1000 Medium
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 1001 - 1500 High
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> 1501 - 2000 Very High
        </div>
    </div>
    """

    # Legend for Soil Aggregate Stability
    legend_html += """
    <div>
        <b>Soil Aggregate Stability</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Poor
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Fair
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Good
        </div>
    </div>
    """

    # Legend for Presence of Invertebrates
    legend_html += """
    <div>
        <b>Presence of Invertebrates</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Poor
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Fair
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Good
        </div>
    </div>
    """

    # Legend for Soil pH
    legend_html += """
    <div>
        <b>Soil pH</b><br>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FF0000; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Acidic (pH &lt; 7)
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Medium (pH 7 - 8.5)
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #00FF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Alkaline (pH 8.51 - 9)
        </div>
        <div style="display: flex; align-items: center; margin: 2px 0;">
            <div style="background: #0000FF; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Fatal (pH &gt; 9)
        </div>
    </div>
    """

    # New Legend for Soil Texture
    legend_html += """
    <div>
    <b>Soil Texture</b><br>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #FFFF00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Silty Clay Loam
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #FF7F00; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Sandy Clay Loam
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #FF1493; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Silty Clay
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #32CD32; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Sandy Clay
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #00FFFF; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Silt Loam
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #FF4500; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Clay Loam
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #8A2BE2; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Clay
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #1E90FF; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Sandy Loam
    </div>
    <div style="display: flex; align-items: center; margin: 2px 0;">
        <div style="background: #FF00FF; width: 12px; height: 12px; margin-right: 5px; border-radius: 50%;"></div> Loamy Sand
    </div>
</div>
"""

    legend_html += "</div>"  # End the legend div
    return legend_html














# Add the custom legend to the map
legend_html = create_property_legends()
m.get_root().html.add_child(folium.Element(legend_html))
















# Add a draggable legend functionality (custom JavaScript)
draggable_script = """
<script>
    var legends = document.querySelectorAll("div[style*='position: fixed']");
    legends.forEach(function(legend) {
        legend.style.cursor = 'move';
        legend.onmousedown = function(e) {
            var shiftX = e.clientX - legend.getBoundingClientRect().left;
            var shiftY = e.clientY - legend.getBoundingClientRect().top;
















            function moveAt(pageX, pageY) {
                legend.style.left = pageX - shiftX + 'px';
                legend.style.top = pageY - shiftY + 'px';
            }
















            moveAt(e.pageX, e.pageY);
















            function onMouseMove(e) {
                moveAt(e.pageX, e.pageY);
            }
















            document.addEventListener('mousemove', onMouseMove);
















            legend.onmouseup = function() {
                document.removeEventListener('mousemove', onMouseMove);
                legend.onmouseup = null;
            };
        };
    });
</script>
"""












# Define the center points for the clusters
jalna_center = [19.8071088, 76.0254723]
wardha_center = [21.0824006, 78.203766]




# Include the link to the Manrope font in the HTML
font_link = '<link href="https://fonts.googleapis.com/css2?family=Manrope:wght@400;600&display=swap" rel="stylesheet">'




# Add a title for the Jalna Cluster with Manrope font
folium.Marker(
    location=jalna_center,
    icon=folium.DivIcon(
        icon_size=(150, 36),  # Adjust size as needed
        icon_anchor=(0, 0),
        html=f'{font_link}<div style="font-family: \'Manrope\', sans-serif; font-size: 18px; font-weight: bold; color: black; background-color: rgba(255, 255, 255, 0.7); padding: 5px; border-radius: 5px;">Jalna Cluster</div>'
    ),
    tooltip="Jalna Cluster"
).add_to(m)




# Add a title for the Wardha Cluster with Manrope font
folium.Marker(
    location=wardha_center,
    icon=folium.DivIcon(
        icon_size=(150, 36),  # Adjust size as needed
        icon_anchor=(0, 0),
        html=f'{font_link}<div style="font-family: \'Manrope\', sans-serif; font-size: 18px; font-weight: bold; color: black; background-color: rgba(255, 255, 255, 0.7); padding: 5px; border-radius: 5px;">Wardha Cluster</div>'
    ),
    tooltip="Wardha Cluster"
).add_to(m)




# Add JavaScript to the map to make legends draggable
m.get_root().html.add_child(folium.Element(draggable_script))








# Set 16:9 aspect ratio for the map size
map_width = '100%'  # Full width
map_height = '56.25vw'  # 16:9 aspect ratio, based on the viewport width (56.25% of the width)




# Inject custom CSS to adjust the size of the map
m.get_root().html.add_child(folium.Element(f"""
    <style>
        .leaflet-container {{
            width: {map_width}px !important;
            height: {map_height}px !important;
        }}
    </style>
"""))














# Display the map
m




<ipython-input-2-3d6e3463653c>:118: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['', '-'], np.nan, inplace=True)


First DataFrame (from Sheet 1):
                  Farmer Name  Farmer code   Latitude  Longitude     Village  \
0        Sachin Diliprao Wagh         8947  20.914808  78.486436  Kharangana   
1  Liladhar Bhomeshwar Galhat         1532  20.949785  78.408013    Chandani   
2     Mangesh Bhimrao Kalokar         6821  20.981018  78.395786     Danapur   
3  Pravin Arunrao asole Asole         6875  21.031581  78.384561  Panjara Bo   
4     Rupesh Gopalrao kalokar         6791  20.991746  78.389724     Danapur   

   Soil organic carbon concentration(%)  \
0                                  0.93   
1                                  0.36   
2                                  0.64   
3                                  0.56   
4                                  0.86   

   Carbon mineralization potential (mgCO2/kg) SRRrecomendation  \
0                                         NaN              NaN   
1                                         NaN              NaN   
2                             

In [ ]:
# Save the map to an HTML file
output_file = "soil_health_map.html"
m.save(output_file)
print(f"Map has been saved as {output_file}")
from google.colab import files
files.download('soil_health_map.html')

Map has been saved as soil_health_map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>